In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_MAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.7 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 221.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 171.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=294850 sha256=96f7ccb1f92ac84f14998ae668d7e6ea518ddab465dd9ee43d874c05f0c0f240
  Stored in directory: /tmp/pip-ephem-wheel-cache-w70_rw1c/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: 

In [8]:
model_name_or_path = 'TheBloke/Llama-2-13B-chat-GGML'
model_basename = 'llama-2-13b-chat.ggmlv3.q5_1.bin'

In [3]:
from huggingface_hub import hf_hub_download

In [5]:
from llama_cpp import Llama

In [18]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [25]:
# GPU

llama = Llama(
    model_path=model_path,
    n_batch=512,  # Adjust according to your VRAM
    n_threads=2,  # Set number of CPU threads
    n_gpu_layers=12  # Adjust as needed
)

llama.cpp: loading model from /root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 9 (mostly Q5_1)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_internal: m

In [26]:
# see the n_layers in GPU
llama.params.n_gpu_layers

12

In [21]:
prompt = 'Write a linear regression in python'
prompt_template = f'''SYSTEM: You are helpful, respectfull and honest assistant. Always answer as helpfull.

USER: {prompt}

ASSISTANT:
'''

In [22]:
response = llama(prompt = prompt_template,
                max_tokens = 3000,
                temperature = 0.5,
                top_p = 0.95,
                repeat_penalty = 1.2,
                top_k = 150,
                echo = True)


llama_print_timings:        load time = 24561.28 ms
llama_print_timings:      sample time =   283.79 ms /   286 runs   (    0.99 ms per token,  1007.79 tokens per second)
llama_print_timings: prompt eval time = 24561.18 ms /    38 tokens (  646.35 ms per token,     1.55 tokens per second)
llama_print_timings:        eval time = 238551.49 ms /   285 runs   (  837.02 ms per token,     1.19 tokens per second)
llama_print_timings:       total time = 264510.25 ms


In [23]:
print(response)

{'id': 'cmpl-22a4bc4a-cd12-452e-a1d8-62c4a96c57a6', 'object': 'text_completion', 'created': 1727676125, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are helpful, respectfull and honest assistant. Always answer as helpfull.\n\nUSER: Write a linear regression in python\n\nASSISTANT:\n\nTo write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:\n```\nfrom sklearn.linear_model import LinearRegression\nimport pandas as pd\n\n# Load your dataset into a Pandas DataFrame\ndf = pd.read_csv('your_data.csv')\n\n# Create a linear regression object and fit the data\nreg = LinearRegression().fit(df[['x1', 'x2']], df['y'])\n\n# Print the coefficients of the linear regression\nprint(reg.coef_)\n```\nThis will output the coefficients of the linear regression, which you can use to make prediction

In [24]:
print(response['choices'][0]['text'])

SYSTEM: You are helpful, respectfull and honest assistant. Always answer as helpfull.

USER: Write a linear regression in python

ASSISTANT:

To write a linear regression in Python, you can use the scikit-learn library. Here is an example of how to do this:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create a linear regression object and fit the data
reg = LinearRegression().fit(df[['x1', 'x2']], df['y'])

# Print the coefficients of the linear regression
print(reg.coef_)
```
This will output the coefficients of the linear regression, which you can use to make predictions on new data. For example:
```
# Make a prediction for a new set of values
new_data = pd.DataFrame({'x1': [2, 3], 'x2': [4, 5]})
prediction = reg.predict(new_data)
print(prediction)
```
This will output the predicted value for the new data.

Please note that this is just a simple example and there are many ot

In [30]:
prompt = 'what is llm and how to fine tune it?'
prompt_template = f'''SYSTEM: You are helpful, respectfull and honest assistant. Always answer as helpfull.

USER: {prompt}

ASSISTANT:
'''

In [31]:
response = llama(prompt = prompt_template,
                max_tokens = 3000,
                temperature = 0.5,
                top_p = 0.95,
                repeat_penalty = 1.2,
                top_k = 150,
                echo = True)

Llama.generate: prefix-match hit

llama_print_timings:        load time = 27282.87 ms
llama_print_timings:      sample time =   331.72 ms /   331 runs   (    1.00 ms per token,   997.84 tokens per second)
llama_print_timings: prompt eval time =  7269.78 ms /    11 tokens (  660.89 ms per token,     1.51 tokens per second)
llama_print_timings:        eval time = 278200.27 ms /   330 runs   (  843.03 ms per token,     1.19 tokens per second)
llama_print_timings:       total time = 287127.87 ms


In [33]:
print(response['choices'][0]['text'])

SYSTEM: You are helpful, respectfull and honest assistant. Always answer as helpfull.

USER: what is llm and how to fine tune it?

ASSISTANT:
LLM stands for Language Model, which is a type of artificial intelligence (AI) model that can be trained on large amounts of text data to generate language outputs that are coherent and natural-sounding. Fine-tuning an LLM involves adjusting its parameters to improve its performance on a specific task or domain.

There are several ways to fine-tune an LLM, including:

1. Task-specific tuning: This involves training the model on a specific task, such as sentiment analysis or question answering, and adjusting its hyperparameters to optimize its performance for that task.
2. Domain adaptation: This involves adapting the model to a new domain or dataset by adding additional data from the target domain during fine-tuning.
3. Transfer learning: This involves using a pre-trained LLM as a starting point and fine-tuning it on a specific task or domain, le